In [1]:
import pandas as pd
import re
import emoji

data = pd.read_csv("twitter_data.csv")

/Users/pgalli/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data.sample(3)

,Unnamed: 0,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,label
15680,15680,814492440803409920,Thu Dec 29 15:25:05 +0000 2016,@madreeve thats not what she said,490044008,1849,561,8229,86352,0,0
19030,19030,813659827356303360,Tue Dec 27 08:16:34 +0000 2016,Pacific Christian Counseling Center https://t....,1458225506,1197,701,1156,61856,0,0
12039,12039,810292910482661376,Sun Dec 18 01:17:39 +0000 2016,"@BOBBi_DEAN I'm happy I know you, even if it's...",18831261,884,2389,8466,57269,0,0


In [3]:
df = data.copy()

In [4]:
df.head()

,Unnamed: 0,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,label
0,0,637894677824413696,Sun Aug 30 07:48:37 +0000 2015,It's just over 2 years since I was diagnosed w...,1013187241,84,211,251,837,0,1
1,1,637890384576778240,Sun Aug 30 07:31:33 +0000 2015,"It's Sunday, I need a break, so I'm planning t...",1013187241,84,211,251,837,1,1
2,2,637749345908051968,Sat Aug 29 22:11:07 +0000 2015,Awake but tired. I need to sleep but my brain ...,1013187241,84,211,251,837,0,1
3,3,637696421077123073,Sat Aug 29 18:40:49 +0000 2015,RT @SewHQ: #Retro bears make perfect gifts and...,1013187241,84,211,251,837,2,1
4,4,637696327485366272,Sat Aug 29 18:40:26 +0000 2015,It’s hard to say whether packing lists are mak...,1013187241,84,211,251,837,1,1


In [5]:
from wordsegment import load, segment
load()

def process_hashtags(tag):
    tag = tag.lstrip('#')
    if not tag:
        return ""
    
    words = segment(tag.lower())
    return ' '.join(words)


def clean_posts(text):
    '''This function is used to clean all the text in the post_text column in the dataframe.
    It elimates emojis, urls, @/mentions, hashtags, numbers, special characters, multiple spaces,
    and converts all letters to lower case.'''
    # Replace emoji's
    text = emoji.replace_emoji(text, replace="")
    
    # Remove any urls
    text = re.sub(r"https/S+|www\S+", "", text)
    
    # Remove @/mentions
    text = re.sub(r"@\w+", "", text)
    
    # Remove hashtags
    text = re.sub(r"#(\w+)", lambda m: process_hashtags(m.group(0)), text)

    
    # Remove numbers
    text = re.sub(r"\d+", "", text)
    
    # Remove special characters/extra puncuation
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    
    # Lower case
    text = text.lower()
    
    # Eliminate multiple spaces
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

In [6]:
df["clean_post"] = df["post_text"].apply(clean_posts)

In [7]:
df.sample(3)

,Unnamed: 0,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,label,clean_post
5801,5801,650851113609113600,Mon Oct 05 01:52:52 +0000 2015,Just got the most unexpected call a few minute...,28727726,629,1975,365,11113,0,1,just got the most unexpected call a few minute...
4315,4315,552166296897732608,Mon Jan 05 18:14:38 +0000 2015,@Nuen_ time to take off the beanie,2285922931,43,59,4301,12309,0,1,time to take off the beanie
15502,15502,815621396596297729,Sun Jan 01 18:11:09 +0000 2017,"The last one is the worst. Today, they'll talk...",490044008,1849,561,8229,86352,3,0,the last one is the worst today theyll talk ab...


In [8]:
df = df[['clean_post']]

In [9]:
df.head()

,clean_post
0,its just over years since i was diagnosed with...
1,its sunday i need a break so im planning to sp...
2,awake but tired i need to sleep but my brain h...
3,rt retro bears make perfect gifts and are grea...
4,its hard to say whether packing lists are maki...


In [10]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

df['clean_post'] = df['clean_post'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

df.head()

,clean_post
0,years since diagnosed anxiety depression today...
1,sunday need break im planning spend little tim...
2,awake tired need sleep brain ideas
3,rt retro bears make perfect gifts great beginn...
4,hard say whether packing lists making life eas...


In [13]:
df.to_csv("twitter_clean_data.csv", index=False)